# 모델 훈련 및 실험 추적

이 노트북에서는 여러 머신러닝 모델을 훈련하고 MLflow를 사용하여 실험을 추적합니다.

## 목표
- 다양한 ML 알고리즘 비교
- 하이퍼파라미터 최적화
- 실험 추적 및 모델 버전 관리
- 최적 모델 선택

In [1]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings('ignore')

# 프로젝트 모듈 임포트
from src.models.trainer import ModelTrainer
from src.utils.experiment_tracker import ExperimentTracker
from src.utils.config import config

print("✅ 라이브러리 및 모듈 임포트 완료")

ImportError: C extension: pandas.util not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

## 1. 데이터 준비

In [ ]:
# 데이터 로드 (실제 프로젝트에서는 전처리된 데이터 사용)
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

print(f"특성 데이터 형태: {X.shape}")
print(f"타겟 데이터 형태: {y.shape}")
print(f"클래스: {data.target_names}")

In [ ]:
# 데이터 분할
trainer = ModelTrainer()
X_train, X_test, y_train, y_test = trainer.prepare_data(X, y)

print(f"훈련 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")
print(f"클래스 분포 (훈련): {y_train.value_counts().sort_index().tolist()}")
print(f"클래스 분포 (테스트): {y_test.value_counts().sort_index().tolist()}")

In [ ]:
# 특성 스케일링
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train), 
    columns=X_train.columns, 
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test), 
    columns=X_test.columns, 
    index=X_test.index
)

print("✅ 특성 스케일링 완료")
print(f"스케일링 후 특성 통계:")
print(X_train_scaled.describe().round(3))

## 2. 실험 추적 설정

In [ ]:
# MLflow 설정
experiment_tracker = ExperimentTracker(use_mlflow=True, use_wandb=False)

# 실험 정보 확인
print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")
print(f"현재 실험: {mlflow.get_experiment_by_name('ml_project_experiments')}")

## 3. 개별 모델 훈련

In [ ]:
# Random Forest 모델 훈련
rf_results = trainer.train_model(
    algorithm='random_forest',
    X_train=X_train_scaled, 
    y_train=y_train,
    X_test=X_test_scaled, 
    y_test=y_test,
    optimize=True
)

print(f"\n🌲 Random Forest 결과:")
print(f"최적 파라미터: {rf_results['best_params']}")
print(f"테스트 정확도: {rf_results['test_metrics']['test_accuracy']:.4f}")

In [ ]:
# XGBoost 모델 훈련
xgb_results = trainer.train_model(
    algorithm='xgboost',
    X_train=X_train_scaled, 
    y_train=y_train,
    X_test=X_test_scaled, 
    y_test=y_test,
    optimize=True
)

print(f"\n🚀 XGBoost 결과:")
print(f"최적 파라미터: {xgb_results['best_params']}")
print(f"테스트 정확도: {xgb_results['test_metrics']['test_accuracy']:.4f}")

## 4. 모든 모델 비교 훈련

In [ ]:
# 모든 설정된 모델 훈련
all_results = trainer.train_all_models(
    X_train=X_train_scaled, 
    y_train=y_train,
    X_test=X_test_scaled, 
    y_test=y_test,
    optimize=True
)

print("\n=== 모든 모델 훈련 완료 ===")
print(f"훈련된 모델 수: {len(all_results)}")

## 5. 결과 비교 및 시각화

In [ ]:
# 모델 성능 비교 테이블
comparison_data = []

for algorithm, results in all_results.items():
    test_metrics = results['test_metrics']
    comparison_data.append({
        'Algorithm': algorithm,
        'Accuracy': test_metrics['test_accuracy'],
        'Precision': test_metrics['test_precision'],
        'Recall': test_metrics['test_recall'],
        'F1-Score': test_metrics['test_f1']
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Accuracy', ascending=False)

print("=== 모델 성능 비교 ===")
display(comparison_df.round(4))

In [ ]:
# 성능 지표 시각화
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
n_metrics = len(metrics)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, metric in enumerate(metrics):
    ax = axes[i]
    bars = ax.bar(comparison_df['Algorithm'], comparison_df[metric], 
                  color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'][:len(comparison_df)])
    
    ax.set_title(f'{metric} 비교', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_ylim(0, 1)
    
    # 값 라벨 추가
    for bar, value in zip(bars, comparison_df[metric]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 레이더 차트로 모델 성능 비교
from math import pi

# 레이더 차트 데이터 준비
categories = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
N = len(categories)

# 각도 계산
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]  # 원을 닫기 위해

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

colors = ['red', 'blue', 'green', 'orange']
for i, (algorithm, results) in enumerate(all_results.items()):
    values = [
        results['test_metrics']['test_accuracy'],
        results['test_metrics']['test_precision'],
        results['test_metrics']['test_recall'],
        results['test_metrics']['test_f1']
    ]
    values += values[:1]  # 원을 닫기 위해
    
    ax.plot(angles, values, 'o-', linewidth=2, label=algorithm, color=colors[i])
    ax.fill(angles, values, alpha=0.25, color=colors[i])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
ax.set_title('모델 성능 레이더 차트', size=16, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax.grid(True)

plt.tight_layout()
plt.show()

## 6. 최적 모델 분석

In [ ]:
# 최적 모델 확인
best_model = trainer.get_best_model()
best_algorithm = comparison_df.iloc[0]['Algorithm']
best_results = all_results[best_algorithm]

print(f"🏆 최적 모델: {best_algorithm}")
print(f"📊 테스트 정확도: {best_results['test_metrics']['test_accuracy']:.4f}")
print(f"⚙️ 최적 파라미터: {best_results['best_params']}")

In [ ]:
# 최적 모델의 예측 결과 분석
y_pred = best_model.predict(X_test_scaled)

# 분류 보고서
print("=== 분류 보고서 ===")
print(classification_report(y_test, y_pred, target_names=data.target_names))

# 혼동 행렬
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=data.target_names, 
            yticklabels=data.target_names)
plt.title(f'{best_algorithm} - 혼동 행렬')
plt.ylabel('실제 클래스')
plt.xlabel('예측 클래스')
plt.tight_layout()
plt.show()

In [ ]:
# 특성 중요도 (가능한 경우)
if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': X_train.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=importance_df, x='importance', y='feature', palette='viridis')
    plt.title(f'{best_algorithm} - 특성 중요도')
    plt.xlabel('중요도')
    plt.ylabel('특성')
    plt.tight_layout()
    plt.show()
    
    print("=== 특성 중요도 순위 ===")
    for idx, row in importance_df.iterrows():
        print(f"{row['feature']}: {row['importance']:.4f}")
else:
    print("선택된 모델은 특성 중요도를 제공하지 않습니다.")

## 7. 실험 결과 저장

In [ ]:
# 훈련 결과 저장
trainer.save_results(all_results, 'experiments/training_results.json')

# 모델 비교 결과 저장
comparison_df.to_csv('experiments/model_comparison.csv', index=False)

print("✅ 실험 결과 저장 완료")
print("📁 저장 위치:")
print("   - experiments/training_results.json")
print("   - experiments/model_comparison.csv")
print("   - models/ (각 모델의 .joblib 파일)")

## 8. MLflow UI 확인

In [ ]:
print("=== MLflow 실험 추적 ===")
print("실험 결과를 확인하려면 터미널에서 다음 명령어를 실행하세요:")
print("")
print("cd ml-project")
print("mlflow ui")
print("")
print("그 후 브라우저에서 http://localhost:5000 으로 접속하세요.")
print("")
print("MLflow UI에서 확인할 수 있는 정보:")
print("- 📊 각 실험의 파라미터와 메트릭")
print("- 📈 메트릭 비교 차트")
print("- 🔧 모델 아티팩트")
print("- 📝 실험 노트")

## 9. 요약 및 다음 단계

In [ ]:
print("=== 모델 훈련 요약 ===")
print(f"🎯 훈련된 모델 수: {len(all_results)}")
print(f"🏆 최고 성능 모델: {best_algorithm}")
print(f"📊 최고 정확도: {comparison_df.iloc[0]['Accuracy']:.4f}")
print(f"🔧 하이퍼파라미터 최적화: ✅ 완료")
print(f"📈 실험 추적: ✅ MLflow 기록")

print("\n=== 다음 단계 권장사항 ===")
print("1. 🔍 모델 해석 및 설명가능성 분석")
print("2. 🧪 교차 검증으로 모델 안정성 검증")
print("3. 🚀 모델 배포 준비")
print("4. 📊 A/B 테스트 설계")
print("5. 🔄 모델 모니터링 시스템 구축")

print("\n=== 성능 개선 아이디어 ===")
print("- 🔧 더 많은 하이퍼파라미터 조합 시도")
print("- 🎯 앙상블 모델 구성")
print("- 📊 더 많은 특성 엔지니어링")
print("- 🔄 교차 검증 전략 개선")
print("- 📈 다른 평가 지표 고려")